<a href="https://colab.research.google.com/github/tubagokhan/ADGM/blob/main/ADGMDocParserV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-docx

In [2]:
'''from docx import Document
import json
import random
import string

# Load the DOCX file
doc = Document("/content/ModelFriendlyCOBS.docx")

# Function to generate a unique 6-digit identifier
def generate_unique_identifier():
    return ''.join(random.choices('ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789', k=6))

# Modified generate_paragraph_number function to handle 'List (a)'
def generate_paragraph_number(style_name, last_number, is_list_a=False, is_description=False, is_description_item=False):
    if is_list_a:
        parts = last_number.split('.')
        if not parts[-1].isdigit():  # If the last part is already a letter
            letter = parts[-1][-1]  # Get the last character, which should be a letter
            next_letter = chr(ord(letter) + 1) if letter != 'z' else 'a'  # Increment the letter
            parts[-1] = parts[-1][:-1] + next_letter  # Replace the last letter with the next letter
        else:
            parts.append('a')  # If the last part is a digit, add 'a'
        return '.'.join(parts)
    elif is_description:
        # Handle Description numbering
        parts = last_number.split('.')
        if len(parts)>2:
          number=str(parts[0])+'.'+parts[1]+'.'+'D'
          return number
        else:
          parts.append('D')
        return '.'.join(parts)
    elif is_description_item:
        # Handle Description numbering
        parts = last_number.split('.')
        if parts[-1]=='D':  # If the last part is already a letter
            parts.append('1')
        else :
            next_number=str(int(parts[-1]) + 1)
            parts[-1] = parts[-1][:-1] + next_number

        return '.'.join(parts)
    else:
        parts = last_number.split('.')
        parts = [part for part in parts if 'D' not in part]
        if style_name == 'Heading 1':
            return str(int(parts[0]) + 1) if parts[0].isdigit() else '1'
        elif style_name == 'Heading 2':
            if len(parts) < 2: parts.append('0')
            parts[0] = str(int(parts[0]))
            parts[1] = str(int(parts[1]) + 1)
            return '.'.join(parts[:2])
        elif style_name in ['Heading 3', 'List Paragraph', 'Normal']:
            if len(parts) < 3: parts.extend(['0'] * (3 - len(parts)))
            parts[0] = str(int(parts[0]))
            parts[1] = str(int(parts[1]))
            parts[2] = str(int(parts[2]) + 1)
            return '.'.join(parts[:3])
        elif style_name == 'Description':
            return '.'.join(parts[:2])

# Function to add a new section with the required structure
def add_new_section(paragraph_number, heading, sections, text=None):
    section = {
        "unique_identifier": generate_unique_identifier(),
        "paragraph_number": paragraph_number,
        "heading": heading,
        "text": text if text else "null",
        "subparagraphs": []
    }
    sections.append(section)
    return section

# Initialize the JSON structure
nested_json = {
    "title": "",
    "DocumentID": generate_unique_identifier(),
    "Chapters": []
}

# Variables to track the current state
current_chapter = None
current_section = None
current_subsection = None
paragraph_number = "0"
paragraph_number_description="0"
temp_the_latest_paragraph_number="0"
temp_the_latest_subsection=None

# Iterate through paragraphs in the DOCX file
for paragraph in doc.paragraphs:
    style_name = paragraph.style.name
    text = paragraph.text.strip()
    if style_name == 'Title':
        nested_json["title"] = text

    elif style_name in ['Heading 1', 'Heading 2', 'Heading 3', 'List Paragraph', 'Normal' ]:
        paragraph_number = generate_paragraph_number(style_name, paragraph_number)
        temp_the_latest_paragraph_number=paragraph_number
        if style_name == 'Heading 1':
            current_chapter = add_new_section(paragraph_number=paragraph_number, heading=text, sections=nested_json["Chapters"])
            current_section = None
            current_subsection = None
        elif style_name == 'Heading 2':
            current_section = add_new_section(paragraph_number=paragraph_number, heading=text, sections=current_chapter["subparagraphs"])
            current_subsection = None
        elif style_name == 'Heading 3':
            current_subsection = add_new_section(paragraph_number=paragraph_number, heading=None, text=text, sections=current_section["subparagraphs"] if current_section else current_chapter["subparagraphs"])
        elif style_name in ['List Paragraph', 'Normal']:
            add_new_section(paragraph_number=paragraph_number, heading=None, sections=current_section["subparagraphs"] if current_section else current_chapter["subparagraphs"], text=text)

    elif style_name == 'Description':
        temp_the_latest_subsection=current_section["subparagraphs"] if current_section else current_chapter["subparagraphs"]
        print("before descption")
        print(temp_the_latest_subsection)
        paragraph_number_description = generate_paragraph_number(style_name, paragraph_number, is_description=True)
        temp_the_latest_paragraph_number=paragraph_number_description
        current_section = add_new_section(paragraph_number=paragraph_number_description, heading=text, sections=current_section["subparagraphs"] if current_section else current_chapter["subparagraphs"], text=None)
        # Keep a reference to the current 'Description' section to add 'Description List Items' as subparagraphs
        last_description_section = current_section

    elif style_name == 'Description List Item':
        # Ensure there's a current 'Description' section to add to
        if last_description_section:
            paragraph_number_description = generate_paragraph_number(style_name, last_number=paragraph_number_description, is_description_item=True)
            temp_the_latest_paragraph_number=paragraph_number_description
            # Add the 'Description List Item' as a subparagraph to the last 'Description' section
            add_new_section(paragraph_number=paragraph_number_description, heading=None, sections=last_description_section["subparagraphs"], text=text)
        else:
            print("Warning: 'Description List Item' found without preceding 'Description' section")


    elif style_name == 'List (a)':
        paragraph_number = generate_paragraph_number(style_name, paragraph_number, is_list_a=True)
        temp_the_latest_paragraph_number=paragraph_number
        if current_subsection:
            add_new_section(paragraph_number=paragraph_number, heading=None, sections=current_subsection["subparagraphs"], text=text)
        elif current_section:
            add_new_section(paragraph_number=paragraph_number, heading=None, sections=current_section["subparagraphs"], text=text)
        elif current_chapter:
            add_new_section(paragraph_number=paragraph_number, heading=None, sections=current_chapter["subparagraphs"], text=text)
# Save the JSON structure to a file
with open("/content/final_nested_cobs.json", "w") as json_file:
    json.dump(nested_json, json_file, indent=4)'''


'from docx import Document\nimport json\nimport random\nimport string\n\n# Load the DOCX file\ndoc = Document("/content/ModelFriendlyCOBS.docx")\n\n# Function to generate a unique 6-digit identifier\ndef generate_unique_identifier():\n    return \'\'.join(random.choices(\'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789\', k=6))\n\n# Modified generate_paragraph_number function to handle \'List (a)\'\ndef generate_paragraph_number(style_name, last_number, is_list_a=False, is_description=False, is_description_item=False):\n    if is_list_a:\n        parts = last_number.split(\'.\')\n        if not parts[-1].isdigit():  # If the last part is already a letter\n            letter = parts[-1][-1]  # Get the last character, which should be a letter\n            next_letter = chr(ord(letter) + 1) if letter != \'z\' else \'a\'  # Increment the letter\n            parts[-1] = parts[-1][:-1] + next_letter  # Replace the last letter with the next letter\n        else:\n            parts.append(\'a\')  # If th

In [3]:
from docx import Document
import json
import random
import string

# Load the DOCX file
doc = Document("/content/ModelFriendlyCOBS.docx")

# Function to generate a unique 6-digit identifier
def generate_unique_identifier():
    return ''.join(random.choices('ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789', k=6))

latest_paragraph_number = "0"  # Initialize a variable to store the latest non-description paragraph number

# Modified generate_paragraph_number function
def generate_paragraph_number(style_name, last_number, is_list_a=False, is_description=False, is_description_item=False):

    if is_list_a:
        parts = last_number.split('.')
        if not parts[-1].isdigit():  # If the last part is already a letter
            letter = parts[-1][-1]  # Get the last character, which should be a letter
            next_letter = chr(ord(letter) + 1) if letter != 'z' else 'a'  # Increment the letter
            parts[-1] = parts[-1][:-1] + next_letter  # Replace the last letter with the next letter
        else:
            parts.append('a')  # If the last part is a digit, add 'a'
        return '.'.join(parts)
    elif is_description:
        # Handle Description numbering
        parts = last_number.split('.')
        if len(parts)>2:
          number=str(parts[0])+'.'+parts[1]+'.'+'D'
          return number
        else:
          parts.append('D')
        return '.'.join(parts)
    elif is_description_item:
        # Handle Description numbering
        parts = last_number.split('.')
        if parts[-1]=='D':  # If the last part is already a letter
            parts.append('1')
        else :
            next_number=str(int(parts[-1]) + 1)
            parts[-1] = parts[-1][:-1] + next_number

        return '.'.join(parts)
    else:
        parts = last_number.split('.')
        parts = [part for part in parts if 'D' not in part]
        if style_name == 'Heading 1':
            return str(int(parts[0]) + 1) if parts[0].isdigit() else '1'
        elif style_name == 'Heading 2':
            if len(parts) < 2: parts.append('0')
            parts[0] = str(int(parts[0]))
            parts[1] = str(int(parts[1]) + 1)
            return '.'.join(parts[:2])
        elif style_name in ['Heading 3', 'List Paragraph', 'Normal']:
            if len(parts) < 3: parts.extend(['0'] * (3 - len(parts)))
            parts[0] = str(int(parts[0]))
            parts[1] = str(int(parts[1]))
            parts[2] = str(int(parts[2]) + 1)
            return '.'.join(parts[:3])
        elif style_name == 'Description':
            return '.'.join(parts[:2])

# Function to add a new section with the required structure
def add_new_section(paragraph_number, heading, sections, text=None):
    section = {
        "unique_identifier": generate_unique_identifier(),
        "paragraph_number": paragraph_number,
        "heading": heading,
        "text": text if text else "null",
        "subparagraphs": []
    }
    sections.append(section)
    return section

# Initialize the JSON structure
nested_json = {
    "title": "",
    "DocumentID": generate_unique_identifier(),
    "Chapters": []
}

# Function to find or create the right spot for the new section based on paragraph_number
def find_or_create_section(paragraph_number, sections):
    parts = paragraph_number.split('.')
    current_level = sections
    for i, part in enumerate(parts[:-1]):  # Navigate through the sections according to the paragraph_number
        found = None
        for section in current_level:
            if section["paragraph_number"] == '.'.join(parts[:i+1]):
                found = section
                break
        if not found:  # If the required subsection doesn't exist, create a new one
            new_section = {
                "unique_identifier": generate_unique_identifier(),
                "paragraph_number": '.'.join(parts[:i+1]),
                "heading": None,
                "text": "null",
                "subparagraphs": []
            }
            current_level.append(new_section)
            found = new_section
        current_level = found["subparagraphs"]
    return current_level

# Initialize the JSON structure
nested_json = {
    "title": "",
    "DocumentID": generate_unique_identifier(),
    "Chapters": []
}

# Iterate through paragraphs in the DOCX file
paragraph_number = "0"
for paragraph in doc.paragraphs:
    style_name = paragraph.style.name
    text = paragraph.text.strip()
    if style_name == 'Title':
        nested_json["title"] = text
    else:
        paragraph_number = generate_paragraph_number(style_name, paragraph_number, is_list_a=style_name == 'List (a)', is_description=style_name == 'Description', is_description_item=style_name == 'Description List Item')
        sections = find_or_create_section(paragraph_number, nested_json["Chapters"])
        # Determine heading and text based on the style
        if style_name in ['Heading 1', 'Heading 2', 'Description']:
            heading = text
            text_content = None
        else:
            heading = None
            text_content = text
        # Add the new section or paragraph
        new_section = {
            "unique_identifier": generate_unique_identifier(),
            "paragraph_number": paragraph_number,
            "heading": heading,
            "text": text_content,
            "subparagraphs": []
        }
        sections.append(new_section)

# Save the JSON structure to a file
with open("/content/final_nested_cobs.json", "w") as json_file:
    json.dump(nested_json, json_file, indent=4)

In [4]:
doc = Document("/content/ModelFriendlyCOBS.docx")

for paragraph in doc.paragraphs:
    print(paragraph.style.name)

Title
Heading 1
Heading 2
Normal
List (a)
List (a)
Normal
Heading 1
Heading 2
Heading 3
Heading 3
Heading 3
List (a)
List (a)
List (a)
List (a)
Description
Description List Item
Description List Item
Description List Item
Heading 2
Heading 3
List (a)
List (a)
Heading 3
Description
Description List Item
Description List Item
Description List Item
Description List Item
Description List Item
Heading 3
Heading 3
